In [1]:
#1. rutin1 import module
import pandas as pd
import os
import sys

In [2]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [3]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r30_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [4]:
# Preprocess
import re

def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()
    text = text.split()

    return text

df_preprocess = df['text'].apply(preprocess)
df_preprocess.head(3)

0    [usulan, personil, penting, proposed, key, per...
1    [template, document, jtb, gpf, project, mengac...
2    [change, inquiry, terkait, usulan, perubahan, ...
Name: text, dtype: object

In [5]:
from gensim.models import Word2Vec

# Combine the tokenized text from 'judul' and 'isi' columns for Word2Vec training
combined_text = df_preprocess.tolist()

# Train the Word2Vec model
model = Word2Vec(sentences=combined_text, vector_size=200, window=5, min_count=1, workers=4)

In [6]:
# Show the vector for a sample word to verify the model has been trained
sample_word = "personil"
if sample_word in model.wv:
    sample_vector = model.wv[sample_word]
    sample_vector[:10]  # Show the first 10 dimensions of the vector
else:
    sample_vector = "The word is not in the model's vocabulary."
sample_vector


array([ 0.29301992, -0.21053556,  0.09474085, -0.06578132,  0.58762074,
       -0.20749418,  0.04658502,  0.6127915 , -0.36693853,  0.07088743,
       -0.09747476, -0.5853474 , -0.02607882,  0.27292496, -0.12088301,
        0.17536394,  0.19788462,  0.01887199, -0.12445644, -0.8273433 ,
        0.28520113, -0.150734  ,  0.25970906, -0.27165663, -0.13289349,
       -0.03076282, -0.34035113, -0.10963814, -0.50268996, -0.21767238,
        0.8021536 ,  0.14931756,  0.55109   , -0.12387231, -0.0769696 ,
       -0.06895958,  0.36246303, -0.14256117, -0.04798238, -0.46314037,
       -0.5317222 , -0.06043896,  0.1056262 ,  0.16585974,  0.28732806,
       -0.09175546, -0.1440714 , -0.01731697,  0.15625148,  0.24834019,
        0.26558793,  0.18306029,  0.05779925, -0.5489578 ,  0.08694011,
       -0.54073817, -0.01016743, -0.53936076, -0.38572463,  0.45442557,
        0.05905569, -0.25443566, -0.21378002,  0.2529141 , -0.48555684,
        0.24832486, -0.03223069,  0.77982175, -0.3431279 ,  0.60

In [7]:
model.save("word2vec_model_surat_lg.model")
